This notebook looks at creating figures for GEDI (Figure 5 A-C)

### Uncertainty Image (Figure 5B)
### Step 1: Import packages

In [ ]:
import ee
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()

import geemap

import geopandas as gpd
from matplotlib.ticker import FuncFormatter
import matplotlib.pyplot as plt

import matplotlib.pyplot as plt
import matplotlib.colorbar as cbar
from matplotlib.colors import Normalize
from matplotlib import cm
from matplotlib.image import imread, imsave

### Step 2: Prepare data

In [ ]:
# Import the canopy height data
ch = ee.ImageCollection("projects/ee-geethensingh/assets/UQ/GEDI_CH100v2")
# Define the band names (by default, automatically uploaded data are given names such as b1, b2, ...)
bandsCH = ee.List(['Canopy height', 'Prediction width', 'lower', 'upper'])
# mosaic the images and rename the band with the list above
ch = ch.mosaic().rename(bandsCH)

# Get the boundary of the first biannual image Planet-NICFI image in africa
boundary = ee.ImageCollection("projects/planet-nicfi/assets/basemaps/africa").filterDate('2020-01-01', '2021-01-01').filter(ee.Filter.eq('cadence', 'biannual')).first()
# Select the african continent
africa = ee.FeatureCollection('projects/ee-geethensingh/assets/World_Continents').filterBounds(ch.geometry())
# mask to remove permanent water bodies
waterMask = ee.ImageCollection("JRC/GSW1_4/YearlyHistory").filterDate('2020-01-01', '2021-01-01').filterBounds(africa).mosaic().eq(3).unmask(0).eq(0)
# Apply masks and clip to continent boundaries
ch = ch.mask(boundary).clipToCollection(africa).updateMask(waterMask).rename(bandsCH)
# Visualise the prediction width band
Map = geemap.Map()
vis_params = {'bands': ['Prediction width'], 'palette': ['#440154', ' #440256', ' #450457', ' #450559', ' #46075a', ' #46085c', ' #460a5d', ' #460b5e', ' #470d60', ' #470e61', ' #471063', ' #471164', ' #471365', ' #481467', ' #481668', ' #481769', ' #48186a', ' #481a6c', ' #481b6d', ' #481c6e', ' #481d6f', ' #481f70', ' #482071', ' #482173', ' #482374', ' #482475', ' #482576', ' #482677', ' #482878', ' #482979', ' #472a7a', ' #472c7a', ' #472d7b', ' #472e7c', ' #472f7d', ' #46307e', ' #46327e', ' #46337f', ' #463480', ' #453581', ' #453781', ' #453882', ' #443983', ' #443a83', ' #443b84', ' #433d84', ' #433e85', ' #423f85', ' #424086', ' #424186', ' #414287', ' #414487', ' #404588', ' #404688', ' #3f4788', ' #3f4889', ' #3e4989', ' #3e4a89', ' #3e4c8a', ' #3d4d8a', ' #3d4e8a', ' #3c4f8a', ' #3c508b', ' #3b518b', ' #3b528b', ' #3a538b', ' #3a548c', ' #39558c', ' #39568c', ' #38588c', ' #38598c', ' #375a8c', ' #375b8d', ' #365c8d', ' #365d8d', ' #355e8d', ' #355f8d', ' #34608d', ' #34618d', ' #33628d', ' #33638d', ' #32648e', ' #32658e', ' #31668e', ' #31678e', ' #31688e', ' #30698e', ' #306a8e', ' #2f6b8e', ' #2f6c8e', ' #2e6d8e', ' #2e6e8e', ' #2e6f8e', ' #2d708e', ' #2d718e', ' #2c718e', ' #2c728e', ' #2c738e', ' #2b748e', ' #2b758e', ' #2a768e', ' #2a778e', ' #2a788e', ' #29798e', ' #297a8e', ' #297b8e', ' #287c8e', ' #287d8e', ' #277e8e', ' #277f8e', ' #27808e', ' #26818e', ' #26828e', ' #26828e', ' #25838e', ' #25848e', ' #25858e', ' #24868e', ' #24878e', ' #23888e', ' #23898e', ' #238a8d', ' #228b8d', ' #228c8d', ' #228d8d', ' #218e8d', ' #218f8d', ' #21908d', ' #21918c', ' #20928c', ' #20928c', ' #20938c', ' #1f948c', ' #1f958b', ' #1f968b', ' #1f978b', ' #1f988b', ' #1f998a', ' #1f9a8a', ' #1e9b8a', ' #1e9c89', ' #1e9d89', ' #1f9e89', ' #1f9f88', ' #1fa088', ' #1fa188', ' #1fa187', ' #1fa287', ' #20a386', ' #20a486', ' #21a585', ' #21a685', ' #22a785', ' #22a884', ' #23a983', ' #24aa83', ' #25ab82', ' #25ac82', ' #26ad81', ' #27ad81', ' #28ae80', ' #29af7f', ' #2ab07f', ' #2cb17e', ' #2db27d', ' #2eb37c', ' #2fb47c', ' #31b57b', ' #32b67a', ' #34b679', ' #35b779', ' #37b878', ' #38b977', ' #3aba76', ' #3bbb75', ' #3dbc74', ' #3fbc73', ' #40bd72', ' #42be71', ' #44bf70', ' #46c06f', ' #48c16e', ' #4ac16d', ' #4cc26c', ' #4ec36b', ' #50c46a', ' #52c569', ' #54c568', ' #56c667', ' #58c765', ' #5ac864', ' #5cc863', ' #5ec962', ' #60ca60', ' #63cb5f', ' #65cb5e', ' #67cc5c', ' #69cd5b', ' #6ccd5a', ' #6ece58', ' #70cf57', ' #73d056', ' #75d054', ' #77d153', ' #7ad151', ' #7cd250', ' #7fd34e', ' #81d34d', ' #84d44b', ' #86d549', ' #89d548', ' #8bd646', ' #8ed645', ' #90d743', ' #93d741', ' #95d840', ' #98d83e', ' #9bd93c', ' #9dd93b', ' #a0da39', ' #a2da37', ' #a5db36', ' #a8db34', ' #aadc32', ' #addc30', ' #b0dd2f', ' #b2dd2d', ' #b5de2b', ' #b8de29', ' #bade28', ' #bddf26', ' #c0df25', ' #c2df23', ' #c5e021', ' #c8e020', ' #cae11f', ' #cde11d', ' #d0e11c', ' #d2e21b', ' #d5e21a', ' #d8e219', ' #dae319', ' #dde318', ' #dfe318', ' #e2e418', ' #e5e419', ' #e7e419', ' #eae51a', ' #ece51b', ' #efe51c', ' #f1e51d', ' #f4e61e', ' #f6e620', ' #f8e621', ' #fbe723', ' #fde725'], 'min': 1.0, 'max': 46.0, 'opacity': 1.0}
Map.addLayer(ch, vis_params, 'Prediction width')
Map


### Step 3: Export visualised image

In [ ]:
# Export visualised prediction width image at 5km resolution
Map.layer_to_image('Prediction width', output=r'C:\Users\coach\myfiles\postdoc\Uncertainty\figures\PW5km.png', region= boundary.geometry(), scale=5000)

PW5km.png.tif: |          | 0.00/7.77M (raw) [  0.0%] in 00:00 (eta:     ?)

In [ ]:
# Visualise exported image
geemap.show_image(r'C:\Users\coach\myfiles\postdoc\Uncertainty\figures\PW5km.png')

### Step 4: Create a publication quality figure

In [ ]:
# Read the PNG image from disk
image_path = r'C:\Users\coach\myfiles\postdoc\Uncertainty\figures\PW5km.png'
image = imread(image_path)

# Plot the image
fig, ax = plt.subplots()
im = ax.imshow(image)
ax.axis('off')  # Turn off the axes

# Add a color bar
cax = fig.add_axes([0.92, 0.2, 0.02, 0.6])  # Adjust the position and size as needed
norm = Normalize(vmin=1, vmax=46)  # Normalize based on the image values
color_map = cm.viridis  # You can choose a different colormap if needed
cb = cbar.ColorbarBase(cax, cmap=color_map, norm=norm)
cb.set_label('Uncertainty (m)', rotation=270, labelpad=15, fontsize=16)  # Add colorbar title
cb.ax.tick_params(labelsize=16)  # Adjust the fontsize of tick labels as needed

# Save the modified image with the color bar
output_path = r'C:\Users\coach\myfiles\postdoc\Uncertainty\figures\PW5km.svg'
fig.savefig(output_path, bbox_inches='tight', pad_inches=0.1, dpi = 350)
plt.close()

### Proportion (uncertainty/canopy height) image (Figure 5C)

In [ ]:
# Compute the uncertainty as a fraction of the canopy height
proportion = ch.select('Prediction width').divide(ch.select('Canopy height')).rename('prop')
# Visualise image
vis_params = {'bands': ['prop'], 'palette': ['#00224e', ' #00234f', ' #002451', ' #002553', ' #002554', ' #002656', ' #002758', ' #002859', ' #00285b', ' #00295d', ' #002a5f', ' #002a61', ' #002b62', ' #002c64', ' #002c66', ' #002d68', ' #002e6a', ' #002e6c', ' #002f6d', ' #00306f', ' #003070', ' #003170', ' #003171', ' #013271', ' #053371', ' #083370', ' #0c3470', ' #0f3570', ' #123570', ' #143670', ' #163770', ' #18376f', ' #1a386f', ' #1c396f', ' #1e3a6f', ' #203a6f', ' #213b6e', ' #233c6e', ' #243c6e', ' #263d6e', ' #273e6e', ' #293f6e', ' #2a3f6d', ' #2b406d', ' #2d416d', ' #2e416d', ' #2f426d', ' #31436d', ' #32436d', ' #33446d', ' #34456c', ' #35456c', ' #36466c', ' #38476c', ' #39486c', ' #3a486c', ' #3b496c', ' #3c4a6c', ' #3d4a6c', ' #3e4b6c', ' #3f4c6c', ' #404c6c', ' #414d6c', ' #424e6c', ' #434e6c', ' #444f6c', ' #45506c', ' #46516c', ' #47516c', ' #48526c', ' #49536c', ' #4a536c', ' #4b546c', ' #4c556c', ' #4d556c', ' #4e566c', ' #4f576c', ' #50576c', ' #51586d', ' #52596d', ' #535a6d', ' #545a6d', ' #555b6d', ' #555c6d', ' #565c6d', ' #575d6d', ' #585e6d', ' #595e6e', ' #5a5f6e', ' #5b606e', ' #5c616e', ' #5d616e', ' #5e626e', ' #5e636f', ' #5f636f', ' #60646f', ' #61656f', ' #62656f', ' #636670', ' #646770', ' #656870', ' #656870', ' #666970', ' #676a71', ' #686a71', ' #696b71', ' #6a6c71', ' #6b6d72', ' #6c6d72', ' #6c6e72', ' #6d6f72', ' #6e6f73', ' #6f7073', ' #707173', ' #717274', ' #727274', ' #727374', ' #737475', ' #747475', ' #757575', ' #767676', ' #777776', ' #777777', ' #787877', ' #797977', ' #7a7a78', ' #7b7a78', ' #7c7b78', ' #7d7c78', ' #7e7c78', ' #7e7d78', ' #7f7e78', ' #807f78', ' #817f78', ' #828079', ' #838179', ' #848279', ' #858279', ' #868379', ' #878478', ' #888578', ' #898578', ' #8a8678', ' #8b8778', ' #8c8878', ' #8d8878', ' #8e8978', ' #8f8a78', ' #908b78', ' #918b78', ' #928c78', ' #928d78', ' #938e78', ' #948e77', ' #958f77', ' #969077', ' #979177', ' #989277', ' #999277', ' #9a9376', ' #9b9476', ' #9c9576', ' #9d9576', ' #9e9676', ' #9f9775', ' #a09875', ' #a19975', ' #a29975', ' #a39a74', ' #a49b74', ' #a59c74', ' #a69c74', ' #a79d73', ' #a89e73', ' #a99f73', ' #aaa073', ' #aba072', ' #aca172', ' #ada272', ' #aea371', ' #afa471', ' #b0a571', ' #b1a570', ' #b3a670', ' #b4a76f', ' #b5a86f', ' #b6a96f', ' #b7a96e', ' #b8aa6e', ' #b9ab6d', ' #baac6d', ' #bbad6d', ' #bcae6c', ' #bdae6c', ' #beaf6b', ' #bfb06b', ' #c0b16a', ' #c1b26a', ' #c2b369', ' #c3b369', ' #c4b468', ' #c5b568', ' #c6b667', ' #c7b767', ' #c8b866', ' #c9b965', ' #cbb965', ' #ccba64', ' #cdbb63', ' #cebc63', ' #cfbd62', ' #d0be62', ' #d1bf61', ' #d2c060', ' #d3c05f', ' #d4c15f', ' #d5c25e', ' #d6c35d', ' #d7c45c', ' #d9c55c', ' #dac65b', ' #dbc75a', ' #dcc859', ' #ddc858', ' #dec958', ' #dfca57', ' #e0cb56', ' #e1cc55', ' #e2cd54', ' #e4ce53', ' #e5cf52', ' #e6d051', ' #e7d150', ' #e8d24f', ' #e9d34e', ' #ead34c', ' #ebd44b', ' #edd54a', ' #eed649', ' #efd748', ' #f0d846', ' #f1d945', ' #f2da44', ' #f3db42', ' #f5dc41', ' #f6dd3f', ' #f7de3e', ' #f8df3c', ' #f9e03a', ' #fbe138', ' #fce236', ' #fde334', ' #fee434', ' #fee535', ' #fee636', ' #fee838'], 'min': 0, 'max': 16.0, 'opacity': 1.0}
Map.addLayer(proportion, vis_params, 'Proportion')
# Export proportion image to a png
Map.layer_to_image('Proportion', output=r'C:\Users\coach\myfiles\postdoc\Uncertainty\figures\UQCH5km.png', region= boundary.geometry(), scale=5000)

UQCH5km.png.tif: |          | 0.00/7.77M (raw) [  0.0%] in 00:00 (eta:     ?)

### Create publication quality uncertainty image for africa
Includes a color bar and is exported at a coarser scale (5km)

In [ ]:
# Read the PNG image from disk
image_path = r'C:\Users\coach\myfiles\postdoc\Uncertainty\figures\UQCH5km.png'
image = imread(image_path)

# Plot the image
fig, ax = plt.subplots()
im = ax.imshow(image)
ax.axis('off')  # Turn off the axes

# Add a color bar
cax = fig.add_axes([0.92, 0.2, 0.02, 0.6])  # Adjust the position and size as needed
norm = Normalize(vmin=0, vmax=16)  # Normalize based on the image values
color_map = cm.cividis  # You can choose a different colormap if needed
cb = cbar.ColorbarBase(cax, cmap=color_map, norm=norm)
cb.set_label('Uncertainty/Canopy height', rotation=270, labelpad=15, fontsize=16)  # Add colorbar title
cb.ax.tick_params(labelsize=16)  # Adjust the fontsize of tick labels as needed

# Save the modified image with the color bar
output_path = r'C:\Users\coach\myfiles\postdoc\Uncertainty\figures\UQCH5km.svg'
fig.savefig(output_path, bbox_inches='tight', pad_inches=0.1, dpi = 350)
plt.close()

### Canopy height (Figure 5A)

In [ ]:
# Visualise image
vis_params = {'bands': ['Canopy height'], 'palette': ['#f7fcf5', ' #f6fcf4', ' #f6fcf4', ' #f5fbf3', ' #f5fbf2', ' #f4fbf2', ' #f4fbf1', ' #f3faf0', ' #f2faf0', ' #f2faef', ' #f1faee', ' #f1faee', ' #f0f9ed', ' #f0f9ec', ' #eff9ec', ' #eff9eb', ' #eef8ea', ' #edf8ea', ' #edf8e9', ' #ecf8e8', ' #ecf8e8', ' #ebf7e7', ' #ebf7e7', ' #eaf7e6', ' #e9f7e5', ' #e9f7e5', ' #e8f6e4', ' #e8f6e3', ' #e7f6e3', ' #e7f6e2', ' #e6f5e1', ' #e5f5e1', ' #e5f5e0', ' #e4f5df', ' #e3f4de', ' #e2f4dd', ' #e1f3dc', ' #e0f3db', ' #dff3da', ' #def2d9', ' #ddf2d8', ' #dcf2d7', ' #dbf1d6', ' #dbf1d5', ' #daf0d4', ' #d9f0d3', ' #d8f0d2', ' #d7efd1', ' #d6efd0', ' #d5efcf', ' #d4eece', ' #d3eecd', ' #d2edcc', ' #d1edcb', ' #d0edca', ' #cfecc9', ' #ceecc8', ' #cdecc7', ' #ccebc6', ' #cbebc5', ' #cbeac4', ' #caeac3', ' #c9eac2', ' #c8e9c1', ' #c7e9c0', ' #c6e8bf', ' #c4e8bd', ' #c3e7bc', ' #c2e7bb', ' #c1e6ba', ' #c0e6b9', ' #bee5b8', ' #bde5b6', ' #bce4b5', ' #bbe4b4', ' #bae3b3', ' #b8e3b2', ' #b7e2b1', ' #b6e2af', ' #b5e1ae', ' #b4e1ad', ' #b2e0ac', ' #b1e0ab', ' #b0dfaa', ' #afdfa8', ' #aedea7', ' #acdea6', ' #abdda5', ' #aadda4', ' #a9dca3', ' #a8dca2', ' #a7dba0', ' #a5db9f', ' #a4da9e', ' #a3da9d', ' #a2d99c', ' #a0d99b', ' #9fd899', ' #9ed798', ' #9cd797', ' #9bd696', ' #99d595', ' #98d594', ' #97d492', ' #95d391', ' #94d390', ' #92d28f', ' #91d28e', ' #90d18d', ' #8ed08b', ' #8dd08a', ' #8bcf89', ' #8ace88', ' #88ce87', ' #87cd86', ' #86cc85', ' #84cc83', ' #83cb82', ' #81ca81', ' #80ca80', ' #7fc97f', ' #7dc87e', ' #7cc87c', ' #7ac77b', ' #79c67a', ' #78c679', ' #76c578', ' #75c477', ' #73c476', ' #72c375', ' #70c274', ' #6ec173', ' #6dc072', ' #6bc072', ' #6abf71', ' #68be70', ' #66bd6f', ' #65bd6f', ' #63bc6e', ' #62bb6d', ' #60ba6c', ' #5eb96b', ' #5db96b', ' #5bb86a', ' #5ab769', ' #58b668', ' #56b567', ' #55b567', ' #53b466', ' #52b365', ' #50b264', ' #4eb264', ' #4db163', ' #4bb062', ' #4aaf61', ' #48ae60', ' #46ae60', ' #45ad5f', ' #43ac5e', ' #42ab5d', ' #40aa5d', ' #3fa95c', ' #3fa85b', ' #3ea75a', ' #3da65a', ' #3ca559', ' #3ba458', ' #3aa357', ' #39a257', ' #38a156', ' #37a055', ' #369f54', ' #359e53', ' #349d53', ' #339c52', ' #329b51', ' #319a50', ' #309950', ' #2f984f', ' #2f974e', ' #2e964d', ' #2d954d', ' #2c944c', ' #2b934b', ' #2a924a', ' #29914a', ' #289049', ' #278f48', ' #268e47', ' #258d47', ' #248c46', ' #238b45', ' #228a44', ' #218944', ' #208843', ' #1f8742', ' #1e8741', ' #1d8640', ' #1c8540', ' #1a843f', ' #19833e', ' #18823d', ' #17813d', ' #16803c', ' #157f3b', ' #147e3a', ' #137d39', ' #127c39', ' #117b38', ' #107a37', ' #0e7936', ' #0d7836', ' #0c7735', ' #0b7734', ' #0a7633', ' #097532', ' #087432', ' #077331', ' #067230', ' #05712f', ' #03702e', ' #026f2e', ' #016e2d', ' #006d2c', ' #006c2c', ' #006b2b', ' #00692a', ' #00682a', ' #006729', ' #006529', ' #006428', ' #006328', ' #006227', ' #006027', ' #005f26', ' #005e26', ' #005c25', ' #005b25', ' #005a24', ' #005924', ' #005723', ' #005622', ' #005522', ' #005321', ' #005221', ' #005120', ' #005020', ' #004e1f', ' #004d1f', ' #004c1e', ' #004a1e', ' #00491d', ' #00481d', ' #00471c', ' #00451c', ' #00441b'], 'min': 0, 'max': 29.0, 'opacity': 1.0}
Map = geemap.Map()
Map.addLayer(ch, vis_params, 'Canopy height')
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Export image
Map.layer_to_image('Canopy height', output=r'C:\Users\coach\myfiles\postdoc\Uncertainty\figures\CH10km.png', region= boundary.geometry(), scale=10000)

CH10km.png.tif: |          | 0.00/1.94M (raw) [  0.0%] in 00:00 (eta:     ?)

In [ ]:
# Visualise exported image
geemap.show_image(r'C:\Users\coach\myfiles\postdoc\Uncertainty\figures\CH10km.png')

Output()

In [ ]:
# Create a publication-ready figure
# Read the PNG image from disk
image_path = r'C:\Users\coach\myfiles\postdoc\Uncertainty\figures\CH10km.png'
image = imread(image_path)

# Plot the image
fig, ax = plt.subplots()
im = ax.imshow(image)
ax.axis('off')  # Turn off the axes

# Add a color bar
cax = fig.add_axes([0.92, 0.2, 0.02, 0.6])  # Adjust the position and size as needed
norm = Normalize(vmin=0, vmax=52)  # Normalize based on the image values
color_map = cm.Greens  # You can choose a different colormap if needed
cb = cbar.ColorbarBase(cax, cmap=color_map, norm=norm)
cb.set_label('Canopy height (m)', rotation=270, labelpad=15, fontsize=16)  # Add colorbar title
cb.ax.tick_params(labelsize=16)  # Adjust the fontsize of tick labels as needed

# Save the modified image with the color bar
output_path = r'C:\Users\coach\myfiles\postdoc\Uncertainty\figures\CH10km.svg'
fig.savefig(output_path, bbox_inches='tight', pad_inches=0.1, dpi = 350)
plt.close()